# Prep Data for Pretraining on T5

In [1]:
from datasets import load_from_disk

dataset = load_from_disk('combined-ds')

dataset = dataset['train'].shuffle().select(range(500_000))

## Corrupt Training Text

T5 is trained by learning to correct missing spans in text. Thus, the training data must have spans masked for training.

In [2]:
import random

def bo_corrupt_text(example):
    text = example["bo"]
    words = text.split()
    num_masks = max(1, len(words) // 6)  # Mask ~15-20% of the words
    masked_indices = sorted(random.sample(range(len(words)), num_masks))

    new_text = []
    labels = []
    current_mask = 0

    for i, word in enumerate(words):
        if i in masked_indices:
            if not new_text or new_text[-1] != f"<extra_id_{current_mask}>":
                new_text.append(f"<extra_id_{current_mask}>")
                labels.append(f"<extra_id_{current_mask}> {word}")
                current_mask += 1
            else:
                labels[-1] += f" {word}"
        else:
            new_text.append(word)

    return {"input_text": " ".join(new_text), "target_text": " ".join(labels)}

bo_train_dataset = dataset.map(bo_corrupt_text)


Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [3]:
def en_corrupt_text(example):
    text = example["en"]
    words = text.split()
    num_masks = max(1, len(words) // 6)  # Mask ~15-20% of the words
    masked_indices = sorted(random.sample(range(len(words)), num_masks))

    new_text = []
    labels = []
    current_mask = 0

    for i, word in enumerate(words):
        if i in masked_indices:
            if not new_text or new_text[-1] != f"<extra_id_{current_mask}>":
                new_text.append(f"<extra_id_{current_mask}>")
                labels.append(f"<extra_id_{current_mask}> {word}")
                current_mask += 1
            else:
                labels[-1] += f" {word}"
        else:
            new_text.append(word)

    return {"input_text": " ".join(new_text), "target_text": " ".join(labels)}

en_train_dataset = dataset.map(en_corrupt_text)

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [4]:
from datasets import concatenate_datasets

ds = concatenate_datasets([en_train_dataset, bo_train_dataset])

In [5]:
ds.save_to_disk('pretraining-ds')

Saving the dataset (0/3 shards):   0%|          | 0/1000000 [00:00<?, ? examples/s]